In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import torch
from transformers import AutoTokenizer
import datasets


all_data_path = '.data/multi30k'
train_en_path = all_data_path + '/train.en'
train_vi_path = all_data_path + '/train.vi'
val_en_path = all_data_path + '/val.en'
val_vi_path = all_data_path + '/val.vi'
test_en_path = all_data_path + '/test2016.en'
test_vi_path = all_data_path + '/test2016.vi'

2024-06-25 15:37:02.785265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 15:37:02.785312: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 15:37:02.785347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-25 15:37:02.798648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 15:37:03.797374: W tensorflow/c

In [2]:
def load_test():
  data = []
  with open(test_en_path) as f1, open(test_vi_path) as f2:
      for src, tgt in zip(f1, f2):
        data.append(
            {
                "translation": {
                    "en": src.strip(),
                    "vi": tgt.strip()
                }
            }
        )
  print(f'total size of data is {len(data)}')
  tdata = pd.DataFrame(data)
  tdata = tdata.reset_index()
  tdata = tdata.rename(columns={'index': 'id'})
  test = datasets.Dataset.from_pandas(tdata)
  return test

def load_train():
    data = []
    with open(train_en_path) as f1, open(train_vi_path) as f2:
        for src, tgt in zip(f1, f2):
          data.append(
              {
                  "translation": {
                      "en": src.strip(),
                      "vi": tgt.strip()
                  }
              }
          )
    print(f'total size of data is {len(data)}')
    tdata = pd.DataFrame(data)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    # don't care about test_size. 
    downsampled_dataset = dataset.train_test_split(
        train_size=20000, test_size=2000, seed=69
    )
        
    return downsampled_dataset
test_set = load_test()
train_set = load_train()

total size of data is 1207
total size of data is 128134


In [3]:
train_set


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2000
    })
})

In [4]:
checkpoint="facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
print(tokenizer(["Hello and i like money!", "i come i see i conquer"]))

{'input_ids': [[250004, 35378, 136, 17, 1884, 17265, 38, 2], [250004, 17, 1380, 17, 1957, 17, 158, 27227, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [5]:
model = MBartForConditionalGeneration.from_pretrained(checkpoint)


In [6]:
max_input_length = 100
max_target_length = 100
source_lang = "en"
target_lang = "vi"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line 
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_datasets = train_set.map(preprocess_function, batched=True)
tokenized_test_set = test_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1207 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

/tmp/ipykernel_99433/985486967.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [9]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [10]:
from accelerate import Accelerator
accelerator = Accelerator()
torch.cuda.empty_cache()
gc.collect()

1519

In [11]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_EnglistToVietnamese/",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
#                         gradient_accumulation_steps=4,
                        learning_rate=2e-5,
                        num_train_epochs=5,
                        predict_with_generate=True,
                        logging_dir="/logs",
                        logging_steps=10000,
                        save_steps=10000,
                        report_to="none"
                        )


trainer = Seq2SeqTrainer(model=model, 
                args=args,
                data_collator=data_collator, 
                train_dataset=tokenized_datasets['train'],
#                eval_dataset=tokenized_datasets['test'],
                 eval_dataset=tokenized_test_set,
                tokenizer=tokenizer,
              compute_metrics=compute_metrics
                )

tokenized_datasets, tokenized_test_set, trainer = accelerator.prepare(
     tokenized_datasets, tokenized_test_set, trainer
      )

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/k64t/_conda/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.314066,33.591800,30.257700
2,No log,1.355187,33.312100,30.251900
3,No log,1.407682,32.791800,30.173200
4,No log,1.481415,32.182300,30.244400
5,No log,1.526963,31.979300,30.205500


TrainOutput(global_step=6250, training_loss=0.805046015625, metrics={'train_runtime': 8946.6471, 'train_samples_per_second': 11.177, 'train_steps_per_second': 0.699, 'total_flos': 1.1469669742411776e+16, 'train_loss': 0.805046015625, 'epoch': 5.0})

In [13]:
from pathlib import Path


def save_model(name):
    MODEL_PATH = Path('models')
    MODEL_NAME = Path(name + '.pth')
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
    #print(f'Saving model to : {MODEL_SAVE_PATH}')
    torch.save(obj = model.state_dict(),
            f = MODEL_SAVE_PATH)

save_model('mbart')